<a href="https://colab.research.google.com/github/RCortez25/PhD/blob/main/LLM/1.%20Data%20Loader/Data_Loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
import tiktoken

In [5]:
url = "https://raw.githubusercontent.com/RCortez25/PhD/main/LLM/0.%20Tokenizer/the-veredict.txt"
response = requests.get(url)

with open("the-veredict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

oTokenizer = tiktoken.get_encoding("gpt2")
encoded_text = oTokenizer.encode(raw_text)
print(f"Number of tokens: {len(encoded_text)}")

Number of tokens: 5146


In [12]:
# Remove the first 50 tokens for demonstration purposes
encoded_sample = encoded_text[50:]

Now, in order to create input-target pairs one creates two variables

$x$: inputs

$y$: targets

For example:

$x=[1,2,3,4]\\y=[2,3,4,5]$

The `context_size` determines how many tokens to include.

In [14]:
context_size = 4

# Creating input-target pair
x = encoded_sample[:context_size]
y = encoded_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


Let's create a simple example of how the LLM will process the text in the autoregressive scheme

In [15]:
for i in range(1, context_size+1):
    context = encoded_sample[:i]
    desired = encoded_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257
